In [1]:
# Bronze Layer - Batch Data Ingestion
# Bu notebook hotel_raw_batch.csv'yi Delta format'a çevirir

print("🚀 Bronze Batch Ingestion başlıyor...")
print(f"Notebook çalışma zamanı: {spark.sparkContext.applicationId}")

StatementMeta(, 7c3ab92a-13c6-40e5-ac56-ab4a0ead42ae, 3, Finished, Available, Finished)

🚀 Bronze Batch Ingestion başlıyor...
Notebook çalışma zamanı: application_1764271934817_0001


In [ ]:
# 🔍 Spark ile CSV okuma - Basit ve Güvenli Yöntem

# Lakehouse bağlıysa bu çalışacak
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/booking/hotel_raw_batch.csv")

print("✅ CSV başarıyla okundu!")
print(f"📊 Toplam satır: {df.count():,}")
print(f"📋 Toplam sütun: {len(df.columns)}")

# İlk 5 satır göster
print("\n👀 İlk 5 satır:")
df.show(5, truncate=False)

In [2]:
# 🔍 Veri Tiplerini ve Kalite Problemlerini İnceleyelim

print("="*80)
print("📋 VERİ TİPLERİ VE YAPISAL ANALİZ")
print("="*80)

# Veri tipi dağılımı
print("\n🔢 Veri Tipleri:")
df.printSchema()

print("\n" + "="*80)
print("❓ EKSİK DEĞER ANALİZİ")
print("="*80)

# Null sayıları
from pyspark.sql.functions import col, count, when, isnan

null_counts = df.select([
    count(when(col(c).isNull() | (col(c) == ''), c)).alias(c) 
    for c in df.columns
]).collect()[0].asDict()

# En çok null olan 20 sütun
import pandas as pd
null_df = pd.DataFrame(list(null_counts.items()), columns=['Column', 'Null_Count'])
null_df['Null_Percentage'] = (null_df['Null_Count'] / df.count() * 100).round(2)
null_df = null_df.sort_values('Null_Count', ascending=False).head(20)

print("\n📉 En Çok Eksik Değer İçeren Sütunlar:")
print(null_df.to_string(index=False))

StatementMeta(, 204c643e-baf6-49ad-ad50-c84573405c49, 4, Finished, Available, Finished)

📋 VERİ TİPLERİ VE YAPISAL ANALİZ

🔢 Veri Tipleri:
root
 |-- hotel_id: string (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- hotel_type: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- total_rooms: string (nullable = true)
 |-- hotel_facilities: string (nullable = true)
 |-- hotel_description: string (nullable = true)
 |-- nearby_attractions: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- website: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country_customer: string (nullable = true)
 |-- city_customer: stri

In [3]:
# 🐛 Problemli Değerleri Tespit Et

from pyspark.sql.functions import col, lit

print("="*80)
print("🔍 SORUNLU DEĞERLER")
print("="*80)

# Placeholder değerler
placeholders = ['___', '???', '--', '...', '9999', 'NULL', '']

# Tarih sütunları kontrol
date_columns = ['birth_date', 'registration_date', 'booking_date', 
                'checkin_date', 'checkout_date', 'cancellation_date', 
                'review_date', 'stay_date', 'created_timestamp', 'updated_timestamp']

print("\n📅 Problemli Tarih Formatları (ilk 10):")
problem_dates = df.select('booking_id', 'booking_date', 'checkin_date', 'checkout_date', 'birth_date') \
    .filter(
        (col('booking_date') == '0000-00-00') | 
        (col('booking_date').contains('13-')) |
        (col('checkin_date').contains('Jan 32')) |
        (col('birth_date').contains('13-'))
    ).limit(10)

problem_dates.show(truncate=False)

# Negatif fiyatlar
print("\n💰 Negatif veya Anormal Fiyatlar:")
df.select('booking_id', 'total_price', 'room_price', 'paid_amount') \
    .filter((col('paid_amount') < 0) | (col('room_price') == 9999) | (col('room_price') == 0)) \
    .show(10)

# Metin değeri olan sayısal alanlar
print("\n🔤 Sayısal Alanlarda Metin (room_booked):")
df.select('booking_id', 'rooms_booked', 'total_rooms', 'star_rating') \
    .filter(col('rooms_booked') == 'Five') \
    .show(10)

print("\n✅ Veri kalite analizi tamamlandı!")

StatementMeta(, 204c643e-baf6-49ad-ad50-c84573405c49, 5, Finished, Available, Finished)

🔍 SORUNLU DEĞERLER

📅 Problemli Tarih Formatları (ilk 10):
+-------------+------------+------------+-------------+----------+
|booking_id   |booking_date|checkin_date|checkout_date|birth_date|
+-------------+------------+------------+-------------+----------+
|BKG_000000001|2024-03-27  |NULL        |2026-01-09   |2023-13-01|
|BKG_000000013|0000-00-00  |2025-05-10  |2025-05-20   |1975-10-19|
|BKG_000000014|2023-13-01  |2024-11-23  |2024-11-24   |1992-04-17|
|BKG_000000026|2025-10-29  |Jan 32, 2025|2025-01-24   |1984-06-10|
|BKG_000000042|0000-00-00  |2026-04-22  |2026-04-24   |2004-10-11|
|BKG_000000176|2023-13-01  |2024-11-25  |2024-12-01   |1946-10-06|
|BKG_000000215|2024-12-21  |2026-06-03  |2026-06-06   |2023-13-01|
|BKG_000000223|2025-03-31  |2025-03-26  |2025-04-08   |2023-13-01|
|BKG_000000289|2024-08-04  |2026-10-20  |!!2026-11-01 |2023-13-01|
|BKG_000000293|2025-07-24  |2026-10-19  |2026-10-30   |2023-13-01|
+-------------+------------+------------+-------------+----------+


💰

In [4]:
# 💾 BRONZE LAYER'A YAZMA (Delta Format)

from pyspark.sql.functions import current_timestamp, lit

print("🚀 Bronze Layer'a yazma başlıyor...")

# Metadata ekle
df_bronze = df.withColumn("ingestion_timestamp", current_timestamp()) \
              .withColumn("source_file", lit("hotel_raw_batch.csv")) \
              .withColumn("layer", lit("bronze"))

# Delta formatında yaz
output_path = "Tables/bronze_hotel_bookings"

df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(output_path)

print(f"✅ Bronze tabloya yazıldı: {output_path}")
print(f"📊 Toplam satır: {df_bronze.count():,}")

# Tablo olarak kaydet
df_bronze.write.format("delta").mode("overwrite").saveAsTable("bronze_hotel_bookings")

print("✅ Delta tablo oluşturuldu: bronze_hotel_bookings")

StatementMeta(, 204c643e-baf6-49ad-ad50-c84573405c49, 6, Finished, Available, Finished)

🚀 Bronze Layer'a yazma başlıyor...
✅ Bronze tabloya yazıldı: Tables/bronze_hotel_bookings
📊 Toplam satır: 1,363,403
✅ Delta tablo oluşturuldu: bronze_hotel_bookings


In [1]:
# Tüm tabloları listele
print("📋 Bronze Lakehouse'daki Tüm Tablolar:")
print("="*50)

tables = spark.catalog.listTables()
for table in tables:
    print(f"  - {table.name}")

StatementMeta(, 260786b2-4d5e-4a74-a297-aba2953ca746, 3, Finished, Available, Finished)

📋 Bronze Lakehouse'daki Tüm Tablolar:
  - bronze_hotel_bookings


In [2]:
# ============================================================================
# VERİ KAYNAĞI ANALİZİ
# ============================================================================

df = spark.table("bronze_hotel_bookings")

print("="*80)
print("📊 BRONZE_HOTEL_BOOKINGS TABLOSU ANALİZİ")
print("="*80)

total_rows = df.count()
print(f"\n✅ Toplam satır: {total_rows:,}")

# Sütunları kontrol et
print(f"\n📋 Sütun sayısı: {len(df.columns)}")
print("\n📝 İlk 5 sütun:")
for col in df.columns[:5]:
    print(f"  - {col}")

# Eventstream'den mi geldi kontrol et
if "event_id" in df.columns:
    print("\n🌊 Bu STREAM verisi! (event_id kolonu var)")
    print(f"   Event sayısı: {df.select('event_id').distinct().count():,}")
    
    # İlk ve son event
    df.select("event_id", "event_timestamp").orderBy("event_timestamp").show(5)
    
elif "ingestion_timestamp" in df.columns:
    print("\n📦 Bu BATCH verisi! (ingestion_timestamp kolonu var)")
    
    # Kaynak dosya kontrolü
    if "source_file" in df.columns:
        df.groupBy("source_file").count().show()
else:
    print("\n❓ Veri kaynağı belirsiz - schema'ya bakalım:")
    df.printSchema()

StatementMeta(, 260786b2-4d5e-4a74-a297-aba2953ca746, 4, Finished, Available, Finished)

📊 BRONZE_HOTEL_BOOKINGS TABLOSU ANALİZİ

✅ Toplam satır: 1,363,403

📋 Sütun sayısı: 75

📝 İlk 5 sütun:
  - hotel_id
  - hotel_name
  - country
  - city
  - hotel_type

📦 Bu BATCH verisi! (ingestion_timestamp kolonu var)
+-------------------+-------+
|        source_file|  count|
+-------------------+-------+
|hotel_raw_batch.csv|1363403|
+-------------------+-------+

